In [13]:
import pandas as pd
import requests
import io

In [14]:
# fetch iris data with request
url = "http://eaton-lab.org/data/iris-data-dirty.csv"
iris_csv=requests.get(url)

In [19]:
iris_df = pd.read_csv(io.StringIO(iris_csv.text), names=["sepal_length","sepal_width", "petal_length", "petal_width", "species_ID"])

In [21]:
iris_df.head()

,sepal_length,sepal_width,petal_length,petal_width,species_ID
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


First, again fix the rows with the misspelled species IDs and remove the rows with NA.

In [29]:
iris_df_clean = iris_df.dropna()

In [25]:
iris_df.loc[:,"species_ID"].unique()

array(['Iris-setosa', 'Iris-setsa', 'Iris-versicolour', 'Iris-versicolor',
       'Iris-virginica'], dtype=object)

In [34]:
iris_df_final = iris_df_clean.copy()

In [35]:
iris_df_final.loc[:, "species_ID"] = iris_df_clean.loc[:,"species_ID"].replace({
     "Iris-setsa": "Iris-setosa",
    "Iris-versicolour": "Iris-versicolor"
})

In [36]:
iris_df_final.loc[:,"species_ID"].unique()

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)

Use the describe() method to show summary statistics over the numerical valued columns. This is cool, but maybe not very intereting because it doesn't show us anything about differences among species.

In [39]:
iris_df_final.describe()

,sepal_length,sepal_width,petal_length,petal_width
count,148.000000,148.000000,148.000000,148.000000
mean,5.841892,3.058108,3.748649,1.195946
std,0.832586,0.434094,1.773635,0.767882
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.575000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


We can split the dataframe into 'groups' based on shared column values using the groupby method. groupby returns an iterable over tuples in the form of (group label, group data). Combine groupby and describe to print summary stats for each of the three species. (Hint: There are many ways to do this, you might use a for loop and unpack the tuples with indexing inside the loop).

In [53]:
for species, stats in iris_df_final.groupby("species_ID"):
    print(species)
    print(stats.describe())

Iris-setosa
       sepal_length  sepal_width  petal_length  petal_width
count      50.00000    50.000000     50.000000     50.00000
mean        5.00600     3.418000      1.464000      0.24400
std         0.35249     0.381024      0.173511      0.10721
min         4.30000     2.300000      1.000000      0.10000
25%         4.80000     3.125000      1.400000      0.20000
50%         5.00000     3.400000      1.500000      0.20000
75%         5.20000     3.675000      1.575000      0.30000
max         5.80000     4.400000      1.900000      0.60000
Iris-versicolor
       sepal_length  sepal_width  petal_length  petal_width
count     48.000000    48.000000     48.000000    48.000000
mean       5.935417     2.770833      4.250000     1.322917
std        0.522062     0.316200      0.469948     0.200255
min        4.900000     2.000000      3.000000     1.000000
25%        5.600000     2.575000      4.000000     1.200000
50%        5.900000     2.800000      4.350000     1.300000
75%        6

 Perform formal tests for differences in group means using ttest_ind from the scipy.stats module (you may need to conda install -c conda-forge scipy first. Hint: You can access the data for a specific group using the get_group() method of the groupby object, e.g. if I called my groupby object gb I could say `gb.get_group("Iris-setosa") and it would return a DataFrame for the data for only these samples).

In [54]:
from scipy import stats

In [60]:
gb = iris_df_final.groupby("species_ID")
iris1=gb.get_group("Iris-setosa")
iris2=gb.get_group("Iris-versicolor")
iris3=gb.get_group("Iris-virginica")

In [64]:
stats.ttest_ind(iris1["petal_length"], iris2["petal_length"], equal_var=False)

Ttest_indResult(statistic=-38.6230374249597, pvalue=1.185853492183689e-43)